# Imports

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps.storymap import JournalStoryMap
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer, Feature
from arcgis.geometry import Geometry, SpatialReference

### Utility Methods

In [2]:
import uuid,json
def getMapAction(fromFeature, forWebmapId, label_field="OBJECTID", label_prefix="Feature Extent"):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}: {1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    ext = Geometry(fromFeature.geometry).extent
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmapId,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": None,
                "popup": None,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<p><a data-storymaps="{0}" data-storymaps-type="media">{1}</a></p>\n'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

### Connect to Portal

In [3]:
prtl = GIS(username='marktorrey')

Enter password: ········


### Get a webmap that has some data we want to query

In [26]:
wm_itm = prtl.content.get('1ad33784a4824ef38f2ce1c60b2963e0')
wm = WebMap(wm_itm)
poly_layer = FeatureLayer(wm.layers[2].url)

### Get the story map

In [28]:
itm = prtl.content.get('876f3dc003184f2ea1d2785e294fd983')
sm = JournalStoryMap(itm)
sections = sm.properties['values']['story']['sections']

### Clear out our test section

In [29]:
# clear out any stuff we tried previously
sections[1]['content'] = '' 
sections[1]['contentActions'] = []
sm.save()

True

### Iterate over our features and generate the map actions

In [30]:
poly_features = poly_layer.query()
for f in poly_features.features:
    ma = getMapAction(f, wm_itm['id'])
    sections[1]['content'] += ma['content_link']
    sections[1]['contentActions'].append(ma['map_action'])
   
print('New Section Content :\n{0}'.format(sections[1]['content']))  
sm.save()


New Section Content :
<p><a data-storymaps="MJ-ACTION-b9e4f821-dbe8-4eee-ab0d-b6c8d4d05fec" data-storymaps-type="media">Feature Extent: 1</a></p>
<p><a data-storymaps="MJ-ACTION-21b1c3fd-374a-4b82-b94f-5ff340c24e9d" data-storymaps-type="media">Feature Extent: 2</a></p>
<p><a data-storymaps="MJ-ACTION-e50a2a69-1aaa-4434-9c1f-c6d598c80eb2" data-storymaps-type="media">Feature Extent: 3</a></p>
<p><a data-storymaps="MJ-ACTION-a41a290f-dd50-4136-93c0-d498eddf550b" data-storymaps-type="media">Feature Extent: 4</a></p>
<p><a data-storymaps="MJ-ACTION-57cc0bb0-c147-4c60-a427-25d1dba808bb" data-storymaps-type="media">Feature Extent: 5</a></p>



True